In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from sklearn.ensemble import AdaBoostRegressor

from warnings import simplefilter

import pickle
from sklearn.externals import joblib


rand_st=4
#simplefilter("ignore") 

In [2]:
#output=pd.read_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')
output=pd.read_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results_robust.xlsx')
output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",342.986713,431.864722,0.458002,0.175,Обучение на сумме рейтингов
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.724529,192.311526,0.574081,0.194,Обучение на разнице в рейтинге


In [3]:
#elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\clean_elos.csv')
elos.head()

,WhiteElo,BlackElo,MeanElos,DiffElos,SumElos
0,2354,2411,2382,-57,4765
1,1915,1999,1957,-84,3914
2,2446,2191,2318,255,4637
3,2168,2075,2121,93,4243
4,2437,2254,2345,183,4691


In [4]:
#features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
features=pd.read_csv(r'C:\Users\vznam\Downloads\PredictingRatings-master\data\features_robust.csv')

print(features.info())
print(features.shape)
features.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Columns: 224 entries, 0 to 223
dtypes: float64(224)
memory usage: 41.5 MB
None
(24255, 224)


,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
count,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,24255.000000,...,24255.000000,24255.000000,24255.000000,24255.00000,24255.000000,24255.000000,24255.00000,24255.000000,24255.000000,24255.000000
mean,0.044877,0.750897,0.009597,0.002444,0.086345,0.827871,-0.834333,46.920092,3.124728,40.735301,...,0.479365,0.465512,0.467697,0.07384,0.472727,-0.368707,0.03892,1.015584,1.457308,0.910740
std,0.417736,5.411864,0.355022,0.091364,1.783087,5.414979,2.465112,128.245270,10.069143,114.569437,...,0.619074,0.502279,0.611065,1.83264,0.502640,0.676858,0.74847,2.874868,4.179615,3.663225
min,-0.500000,-10.000000,-3.321955,-2.349066,-9.460765,-10.000000,-14.229961,-11.664600,-7.688687,-33.209975,...,0.000000,0.000000,0.000000,-4.00000,0.000000,-1.000000,-1.00000,-1.000000,0.000000,0.000000
25%,-0.500000,-3.000000,-0.129611,0.000000,-0.279324,-3.000000,-0.471897,-5.015215,-2.981327,-20.309038,...,0.000000,0.000000,0.000000,-1.00000,0.000000,-1.000000,-1.00000,-1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,0.500000,3.666667,0.151033,0.005336,0.421687,3.833333,0.066089,30.654795,5.491919,35.106446,...,1.000000,1.000000,1.000000,1.00000,1.000000,0.000000,1.00000,2.000000,0.000000,0.000000
max,0.500000,41.666667,3.112170,2.310486,9.519408,41.666667,0.117105,687.253466,115.330300,919.718217,...,2.000000,2.000000,2.000000,4.00000,2.000000,1.000000,1.00000,70.000000,35.000000,36.000000


In [5]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [6]:
X_train_sum, X_test_sum, Y_train_sum, Y_test_sum =ms.train_test_split(features, target_sum, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_sum), len(Y_train_sum)))
print(X_train_sum.shape)
print(Y_train_sum.shape)
print(X_test_sum.shape)
print(Y_test_sum.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


In [7]:
X_train_diff, X_test_diff, Y_train_diff, Y_test_diff =ms.train_test_split(features, target_diff, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train_diff), len(Y_train_diff)))
print(X_train_diff.shape)
print(Y_train_diff.shape)
print(X_test_diff.shape)
print(Y_test_diff.shape)

Train data size: 16978 instances 
Test data size: 16978 instances
(16978, 224)
(16978,)
(7277, 224)
(7277,)


#### Для теста

In [9]:
target_sum1=target_sum[:5000]
target_diff1=target_diff[:5000]
features1=features[:5000]

X_train_sum1, X_test_sum1, Y_train_sum1, Y_test_sum1 =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)

In [10]:
X_train_diff1, X_test_diff1, Y_train_diff1, Y_test_diff1 =ms.train_test_split(features1, target_diff1, test_size = 0.3,random_state=rand_st)

In [11]:
def predictions_report(pred_target,predictions):
    mae_report=mae(pred_target,predictions)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(pred_target, predictions))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(pred_target,predictions)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [12]:
def GridTune_sum(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    
    grid_best_model = grid.fit(X_train_sum1,Y_train_sum1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [13]:
ada_sum_params =[{'n_estimators': [20,50, 100,200,500],'learning_rate':[0.5,1,1.5,2],'loss':['linear', 'square', 'exponential']}]

start_ada_sum = time.time()

ada_sum=AdaBoostRegressor(random_state=rand_st)
ada_sum_model=GridTune_sum(ada_sum_params,ada_sum,5)
ada_sum_predictions=ada_sum_model.predict(X_test_sum1)

scores_ada_sum=predictions_report(Y_test_sum1,ada_sum_predictions)
full_time_ada_sum=round(time.time() - start_ada_sum,3)
print(full_time_ada_sum, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'learning_rate': 2, 'loss': 'square', 'n_estimators': 500}

Лучший алгоритм:
AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',
         n_estimators=500, random_state=4)
MAE = 416.054
RMSE = 536.738
Correlation coefficient = 0.359
416.516 c


In [14]:
start_ada_sum = time.time()

ada_sum=AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',
         n_estimators=500, random_state=4).fit(X_train_sum,Y_train_sum)
ada_sum_predictions=ada_sum.predict(X_test_sum)

scores_ada_sum=predictions_report(Y_test_sum,ada_sum_predictions)
full_time_ada_sum=round(time.time() - start_ada_sum,3)
print(full_time_ada_sum, 'c')

MAE = 359.781
RMSE = 450.024
Correlation coefficient = 0.386
95.62 c


In [16]:
output = output.append(pd.Series([str(ada_sum), scores_ada_sum[0],scores_ada_sum[1], 
                                  scores_ada_sum[2][0],full_time_ada_sum,'AdaBoost по сумме рейтингов'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",186.539067,234.559134,0.486111,0.367,Обучение на полном набооре -> прогноз рейтинго...
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",189.358443,237.420124,0.483303,0.367,Обучение на полном набооре -> прогноз рейтинго...
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",175.593949,221.353796,0.453718,0.164,Обучение только на победах белых -> прогноз ре...
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",176.326439,221.767050,0.461520,0.269,Обучение только на победах черных -> прогноз р...
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",177.431734,224.595199,0.464714,0.151,Обучение только на ничьих -> прогноз ретйтинга...
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",182.048649,228.384992,0.451721,0.142,Обучение только на ничьих -> прогноз ретйтинга...
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",192.062805,241.580990,0.467338,0.151,Обучение только на проигрышах белых -> прогноз...
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",193.242293,242.404488,0.461618,0.165,Обучение только на проигрышах черных -> прогно...
8,"LinearRegression(copy_X=True, fit_intercept=Tr...",171.079101,215.522294,0.461243,0.370,Обучение на среднем ЭЛО
9,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.700896,192.272077,0.574320,0.362,Обучение на разнице в рейтинге


In [17]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [31]:
with open('ada_sum.pkl', 'wb') as f:
    pickle.dump(ada_sum,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(ada_sum, 'ada_sum1.pkl')
#model_sum= joblib.load('filename.pkl') 

['ada_sum1.pkl']

In [18]:
def GridTune_diff(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,
                           refit=True,  
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))
    
    grid_best_model = grid.fit(X_train_diff1,Y_train_diff1)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [23]:
ada_diff_params =[{'n_estimators': [20,50, 100,200,500],'learning_rate':[0.5,1,1.5,2],'loss':['linear', 'square', 'exponential']}]

start_ada_diff = time.time()

ada_diff=AdaBoostRegressor(random_state=rand_st)
ada_diff_model=GridTune_diff(ada_diff_params,ada_diff,5)
ada_diff_predictions=ada_diff_model.predict(X_test_diff1)

scores_ada_diff=predictions_report(Y_test_diff1,ada_diff_predictions)
full_time_ada_diff=round(time.time() - start_ada_diff,3)
print(full_time_ada_diff, 'c')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Лучшие параметры для MAE:
{'learning_rate': 2, 'loss': 'square', 'n_estimators': 50}

Лучший алгоритм:
AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',
         n_estimators=50, random_state=4)
MAE = 154.325
RMSE = 195.463
Correlation coefficient = 0.521
399.55 c


In [26]:
start_ada_diff = time.time()

ada_diff=AdaBoostRegressor(base_estimator=None, learning_rate=2, loss='square',
         n_estimators=50, random_state=4).fit(X_train_diff,Y_train_diff)
ada_diff_predictions=ada_diff.predict(X_test_diff)

scores_ada_diff=predictions_report(Y_test_diff,ada_diff_predictions)
full_time_diff_sum=round(time.time() - start_ada_diff,3)
print(full_time_ada_diff, 'c')

MAE = 160.557
RMSE = 204.833
Correlation coefficient = 0.528
399.55 c


In [27]:
output = output.append(pd.Series([str(ada_diff), scores_ada_diff[0],scores_ada_diff[1], 
                                  scores_ada_diff[2][0],full_time_ada_diff,'AdaBoost на разнице в рейтинге'],
                                 index=output.columns ),
                      ignore_index=True)

output

,Модель,MAE,RMSE,Коэффициент корр. Пирсона,Время работы (c),Комментарий
0,"LinearRegression(copy_X=True, fit_intercept=Tr...",186.539067,234.559134,0.486111,0.367,Обучение на полном набооре -> прогноз рейтинго...
1,"LinearRegression(copy_X=True, fit_intercept=Tr...",189.358443,237.420124,0.483303,0.367,Обучение на полном набооре -> прогноз рейтинго...
2,"LinearRegression(copy_X=True, fit_intercept=Tr...",175.593949,221.353796,0.453718,0.164,Обучение только на победах белых -> прогноз ре...
3,"LinearRegression(copy_X=True, fit_intercept=Tr...",176.326439,221.767050,0.461520,0.269,Обучение только на победах черных -> прогноз р...
4,"LinearRegression(copy_X=True, fit_intercept=Tr...",177.431734,224.595199,0.464714,0.151,Обучение только на ничьих -> прогноз ретйтинга...
5,"LinearRegression(copy_X=True, fit_intercept=Tr...",182.048649,228.384992,0.451721,0.142,Обучение только на ничьих -> прогноз ретйтинга...
6,"LinearRegression(copy_X=True, fit_intercept=Tr...",192.062805,241.580990,0.467338,0.151,Обучение только на проигрышах белых -> прогноз...
7,"LinearRegression(copy_X=True, fit_intercept=Tr...",193.242293,242.404488,0.461618,0.165,Обучение только на проигрышах черных -> прогно...
8,"LinearRegression(copy_X=True, fit_intercept=Tr...",171.079101,215.522294,0.461243,0.370,Обучение на среднем ЭЛО
9,"LinearRegression(copy_X=True, fit_intercept=Tr...",150.700896,192.272077,0.574320,0.362,Обучение на разнице в рейтинге


In [28]:
#output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)
output.to_excel(r'C:\Users\vznam\Downloads\PredictingRatings-master\models\results.xlsx',index=False)

In [32]:
with open('ada_diff.pkl', 'wb') as f:
    pickle.dump(ada_diff,f) 
                         
'''rf_sum = pickle.loads(model_rf_sum)
rf_sum.predict(X[0])'''

joblib.dump(ada_diff, 'ada_diff1.pkl')
#model_sum= joblib.load('filename.pkl') 

['ada_diff1.pkl']

In [35]:
white_elos = (ada_sum_predictions + ada_diff_predictions)/2
black_elos = (ada_sum_predictions-ada_diff_predictions)/2
pred_elos=list(zip(white_elos,black_elos))
pred_elos[:5],elos[:5]

([(2105.61161811668, 2222.592595821732),
  (2222.550923507988, 2358.10862559238),
  (2242.2160239257096, 2359.197001630762),
  (2167.527914707539, 2160.676299230873),
  (2112.072660975574, 2216.8131750514613)],
    WhiteElo  BlackElo  MeanElos  DiffElos  SumElos
 0      2354      2411      2382       -57     4765
 1      1915      1999      1957       -84     3914
 2      2446      2191      2318       255     4637
 3      2168      2075      2121        93     4243
 4      2437      2254      2345       183     4691)

In [36]:
def find_mean_error(pred,elos):
    er=0
    count=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
        count+=2
    return round(er/count,5)

def find_abs_error(pred,elos):
    er=0
    for ind, val in enumerate(pred):
        er+=elos.iloc[ind,0]-val[0]
        er+=elos.iloc[ind,1]-val[1]
    return round(er,5)

In [37]:
find_abs_error(pred_elos,elos),find_mean_error(pred_elos,elos)

(166462.04944, 11.43755)